In [9]:
import polars as pl
import pandas as pd 
import os
import re
import glob
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile
from rarfile import RarFile
from bs4 import BeautifulSoup
import difflib
from br_denatran_frota.code.utils import *
from br_denatran_frota.code.constants import DICT_UFS, SUBSTITUTIONS

In [ ]:

#treat_uf_tipo('../files/2022/frota_por_uf_e_tipo_de_veículo_2-2022.xls')

In [ ]:
from download_frota import download_frota

In [ ]:
download_frota(2, 2022)

Download of frota_por_uf_e_tipo_de_veículo_2-2022.xls complete
Download of frota_por_município_e_tipo_2-2022.xls complete


In [10]:
import basedosdados as bd 
municipios_query = """SELECT nome, id_municipio, sigla_uf FROM `basedosdados.br_bd_diretorios_brasil.municipio`
"""
bd_municipios = bd.read_sql(municipios_query, 'tamir-pipelines')
bd_municipios = pl.from_pandas(bd_municipios)

/home/tamir/.cache/pypoetry/virtualenvs/pipelines-Fyk5jlG0-py3.10/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.1.0)/charset_normalizer (2.0.8) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
Downloading: 100%|██████████| 5570/5570 [00:00<00:00, 8581.01rows/s]


In [11]:
file = '../files/2022/frota_por_município_e_tipo_2-2022.xls'
filename = os.path.split(file)[1]
df = pd.read_excel(file)
new_df  = change_df_header(df, guess_header(df))
new_df.rename(columns={ new_df.columns[0]: "sigla_uf" , new_df.columns[1]: 'nome_denatran'}, inplace= True) # Rename for ease of use.
new_df.sigla_uf = new_df.sigla_uf.str.strip() # Remove whitespace.
new_df = pl.from_pandas(new_df)

In [ ]:
new_df = new_df.with_columns(new_df.apply(fix_suggested_nome_ibge)['apply'].alias('suggested_nome_ibge'))

In [ ]:
# for uf in DICT_UFS:
denatran_uf = new_df.filter(pl.col('sigla_uf') == uf)
ibge_uf = bd_municipios.filter(pl.col('sigla_uf') == uf)
ibge_uf = ibge_uf.with_columns(pl.col('nome').str.to_lowercase())

In [ ]:

x = denatran_uf['nome_denatran'].apply(get_city_name_ibge)
denatran_uf = denatran_uf.with_columns(x.alias('suggested_nome_ibge'))

In [4]:
match_ibge(denatran_uf, ibge_uf)

NameError: name 'denatran_uf' is not defined

In [ ]:
nomes = ibge_uf['nome'].to_list()

In [ ]:
joined_df = denatran_uf.join(ibge_uf, left_on=['suggested_nome_ibge', 'sigla_uf'], right_on=['nome', 'sigla_uf'], how='left')

mismatched = joined_df.filter(pl.col('id_municipio').is_null())

In [ ]:
zdf.loc[zdf['nome_denatran'] == 'ASSU']

,sigla_uf,nome_denatran,TOTAL,AUTOMOVEL,BONDE,CAMINHAO,CAMINHAO TRATOR,CAMINHONETE,CAMIONETA,CHASSI PLATAF,...,QUADRICICLO,REBOQUE,SEMI-REBOQUE,SIDE-CAR,OUTROS,TRATOR ESTEI,TRATOR RODAS,TRICICLO,UTILITARIO,suggested_nome_ibge
11,RN,ASSU,26667,8400,0,960,57,1668,322,0,...,0,219,97,0,97,0,0,12,186,açu
